<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 9.6: Sentiment Analysis
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

Based on the video tutorial **Text Classification with Machine Learning,SpaCy and Scikit(Sentiment Analysis)** by **Jesse E. Agbe (JCharis)**.

## Data Source: UCI
### UCI - Machine Learning Repository
- Center for Machine Learning and Intelligent Systems

The [**UCI Machine Learning Repository**](http://archive.ics.uci.edu/ml/about.html) is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.

### Dataset
- [Sentiment Labelled Sentences Data Set](http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

### Abstract
The dataset contains sentences labelled with positive or negative sentiment.

- Data Set Characteristics: Text
- Number of Instances: 3000
- Area: N/A
- Attribute Characteristics: N/A
- Number of Attributes: N/A
- Date Donated: 2015-05-30
- Associated Tasks: Classification
- Missing Values? N/A
- Number of Web Hits: 102584

### Source
Dimitrios Kotzias dkotzias '@' ics.uci.edu

### Data Set Information
This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015

Please cite the paper if you want to use it :)

It contains sentences labelled with positive or negative sentiment.

### Format
sentence &lt;tab&gt; score &lt;newline&gt;

### Details
Score is either 1 (for positive) or 0 (for negative)

The sentences come from three different websites/fields:
- imdb.com
- amazon.com
- yelp.com

For each website, there exist **500 positive** and **500 negative** sentences. Those were selected randomly for larger datasets of reviews.

We attempted to select sentences that have a clearly positive or negative connotaton, the goal was for no neutral sentences to be selected.

For the full datasets look:

- **imdb**: Maas et. al., 2011 _Learning word vectors for sentiment analysis_
- **amazon**: McAuley et. al., 2013 _Hidden factors and hidden topics: Understanding rating dimensions with review text_
- **yelp**: [Yelp dataset challenge](http://www.yelp.com/dataset_challenge)


### Attribute Information
The attributes are text sentences, extracted from reviews of products, movies, and restaurants

### Relevant Papers
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

### Citation Request
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

## Import libraries

In [2]:
## Import Libraries
import pandas as pd

import regex as re
import spacy

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings('ignore')

## Load data

Load Yelp, Amazon and Imdb Data.

Hint: Source is separated by <tab>s and has no headers.

In [3]:
# source is separated by <tab>s and has no headers
yelp = pd.read_csv('../DATA/yelp_labelled.txt', header = None, sep = '\t')
imdb = pd.read_csv('../DATA/imdb_labelled_fixed.txt', header = None, sep = '\t') # New file without double quotes
amazon = pd.read_csv('../DATA/amazon_cells_labelled.txt', header = None, sep = '\t')

## Inspect the data

Check your datasets.

In [4]:
yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [6]:
imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


## Merge the data

Merge all three datasets.

In [7]:
#Merge and rename columns:

merged = pd.concat([amazon,yelp,imdb])
merged = merged.rename(columns = {0: 'text',1:'target'})

In [8]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3000 non-null   object
 1   target  3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


## Export the data

Export merged datasets to as csv file.

In [9]:
merged.to_csv('../Data/Merged_reviews.csv', index = None)

## Prepare the stage
- Load spaCy

In [10]:
nlp = spacy.load('en_core_web_sm')

## Prepare the text
All the text handling and preparation concerned with the changes and modifications from the raw source text to a format that will be used for the actual processing, things like:
- handle encoding
- handle extraneous and international charaters
- handle simbols
- handle metadata and embeded information
- handle repetitions (such multiple spaces or newlines)

Clean text.

In [11]:
def clean_text(text):
    # reduce multiple spaces and newlines to only one
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
    # remove double quotes
    text = re.sub(r'"', '', text)

    return text

In [12]:
#Apply function to text:
merged['text'] = merged['text'].apply(clean_text)

## Work the text
Concern with the meaning and the substance of the content to extract actual information.

Hint: Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.

In [13]:
#Try testing on an example:

text = merged.iloc[0]['text']
text

'So there is no way for me to plug it in here in the US unless I go by a converter.'

In [14]:
#Remove stopwords:

st = nlp(text)

stop_removed = [i for i in st if not i.is_stop] 
stop_removed

[way, plug, converter, .]

In [15]:
#Remove punctuations:

stop_punc_removed = [i for i in stop_removed if not i.is_punct]
stop_punc_removed

[way, plug, converter]

In [16]:
''.join([str(i) for i in stop_punc_removed])

'wayplugconverter'

In [17]:
#Combine both into a function:

def tokenize(text):
    st = nlp(text)
    stop_removed = [i for i in st if not i.is_stop] 
    stop_punc_removed = [i for i in stop_removed if not i.is_punct]
    combined = ' '.join([str(i) for i in stop_punc_removed])
    
    return combined

In [18]:
#Tokenize the text column:

merged['text'] = merged['text'].apply(tokenize)

In [19]:
merged.head()

,text,target
0,way plug converter,0
1,Good case Excellent value,1
2,Great jawbone,1
3,Tied charger conversations lasting 45 minutes ...,0
4,mic great,1


## Modelling

In [20]:
# helper function to show results and charts
def show_summary_report(actual, prediction):

    if isinstance(actual, pd.Series):
        actual = actual.values
    if actual.dtype.name == 'object':
        actual = actual.astype(int)
    if prediction.dtype.name == 'object':
        prediction = prediction.astype(int)

    accuracy_ = accuracy_score(actual, prediction)
    precision_ = precision_score(actual, prediction)
    recall_ = recall_score(actual, prediction)
    roc_auc_ = roc_auc_score(actual, prediction)

    print('Accuracy : %.4f [TP / N] Proportion of predicted labels that match the true labels. Best: 1, Worst: 0' % accuracy_)
    print('Precision: %.4f [TP / (TP + FP)] Not to label a negative sample as positive.        Best: 1, Worst: 0' % precision_)
    print('Recall   : %.4f [TP / (TP + FN)] Find all the positive samples.                     Best: 1, Worst: 0' % recall_)
    print('ROC AUC  : %.4f                                                                     Best: 1, Worst: < 0.5' % roc_auc_)
    print('-' * 107)
    print('TP: True Positives, FP: False Positives, TN: True Negatives, FN: False Negatives, N: Number of samples')

    # Confusion Matrix
    mat = confusion_matrix(actual, prediction)

    # Precision/Recall
    precision, recall, _ = precision_recall_curve(actual, prediction)
    average_precision = average_precision_score(actual, prediction)
    
    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(actual, prediction)
    roc_auc = auc(fpr, tpr)

    # plot
    fig, ax = plt.subplots(1, 3, figsize = (18, 6))
    fig.subplots_adjust(left = 0.02, right = 0.98, wspace = 0.2)

    # Confusion Matrix
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False, cmap = 'Blues', ax = ax[0])

    ax[0].set_title('Confusion Matrix')
    ax[0].set_xlabel('True label')
    ax[0].set_ylabel('Predicted label')
    
    # Precision/Recall
    step_kwargs = {'step': 'post'}
    ax[1].step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
    ax[1].fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
    ax[1].set_ylim([0.0, 1.0])
    ax[1].set_xlim([0.0, 1.0])
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('2-class Precision-Recall curve')

    # ROC
    ax[2].plot(fpr, tpr, color = 'darkorange', lw = 2, label = 'ROC curve (AUC = %0.2f)' % roc_auc)
    ax[2].plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    ax[2].set_xlim([0.0, 1.0])
    ax[2].set_ylim([0.0, 1.0])
    ax[2].set_xlabel('False Positive Rate')
    ax[2].set_ylabel('True Positive Rate')
    ax[2].set_title('Receiver Operating Characteristic')
    ax[2].legend(loc = 'lower right')

    plt.show()
    
    return (accuracy_, precision_, recall_, roc_auc_)

## Prepare X and y variables:

In [47]:
X = merged['text']
y = merged['target']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Use Bag-of-Words

In [68]:
# Initiate word counter and transform data
vect = CountVectorizer()

countmatrix = vect.fit_transform(X_train,y_train)

In [69]:
#Count matrix is a sparse matrix type
type(countmatrix)

scipy.sparse.csr.csr_matrix

In [70]:
#Choose and fit a logistic regression as the model:

from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(countmatrix, y_train)

LogisticRegression()

In [71]:
#Make predictions for test data:

count_predictions = logmodel.predict(vect.transform(X_test))

In [72]:
#Check test accuracy:

from sklearn.metrics import accuracy_score

print('Log Model accuracy:',accuracy_score(y_test,predictions))

Log Model accuracy: 0.8131313131313131


## Repeat using TF-IDF
TF-IDF = Term Frequency - Inverse Document Frequency

In [73]:
# Initiate word counter and transform data
tfidf_vect = TfidfVectorizer()

tfidf_countmatrix = tfidf_vect.fit_transform(X_train,y_train)

In [74]:
#Count matrix is a sparse matrix type
type(tfidf_countmatrix)

scipy.sparse.csr.csr_matrix

In [75]:
#Choose and fit a logistic regression as the model:

from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(tfidf_countmatrix, y_train)

LogisticRegression()

In [76]:
#Make predictions for test data:

tfidf_predictions = logmodel.predict(tfidf_vect.transform(X_test))

In [77]:
#Check test accuracy:

from sklearn.metrics import accuracy_score

print('Log Model accuracy:',accuracy_score(y_test,predictions))

Log Model accuracy: 0.8131313131313131


## Repeating it all for comparision
Repeating the whole lot in one big block

Find 'Accuracy', 'Precision', 'Recall', 'ROC_AUC' using CountVectorizer and TfidfVectorizer and keep the result in a dataframe.

In [81]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score

models = ['Count Vectorizer', 'Tf-idf vectorizer']
predictions = [count_predictions, tfidf_predictions]
accuracy = [accuracy_score(y_test,i) for i in predictions]
precision = [precision_score(y_test,i) for i in predictions]
roc_auc = [roc_auc_score(y_test,i) for i in predictions]

In [99]:
df = pd.DataFrame([models,accuracy,precision,roc_auc]).transpose()
columnnames = ['models','accuracy','precision','roc_auc']
df.rename(columns = {i:columnnames[i] for i in range(4)})

,models,accuracy,precision,roc_auc
0,Count Vectorizer,0.809091,0.804878,0.807514
1,Tf-idf vectorizer,0.813131,0.792453,0.813221


© 2020 Institute of Data